In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [9]:
np.random.seed(42)
#read the data from the csv file
train_data = pd.read_csv('D:/kaggleCompetition/LeafClassification/train.csv/train.csv')
train_data.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [17]:
train_data.shape
train_x = train_data.drop(['id','species'],axis=1).values

In [59]:
train_x.shape

(990L, 192L)

In [61]:
#data vector
le = LabelEncoder().fit(train_data['species'])
train_y = le.transform(train_data['species'])
train_y.shape

(990L,)

In [48]:
#standard scaler the data
scaler = StandardScaler().fit(train_x)
train_x = scaler.transform(train_x)

In [49]:
params = {'C':[100,1000],'tol':[0.001,0.0001]}
log_reg = LogisticRegression(solver='lbfgs',multi_class='multinomial')
#tune the params
log_clf = GridSearchCV(log_reg,params,scoring = 'log_loss',refit='true',n_jobs=-1,cv=5)
log_clf.fit(train_x,train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [100, 1000], 'tol': [0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit='true', scoring='log_loss',
       verbose=0)

In [50]:
print("best params: " + str(log_clf.best_params_))
for params, mean_score, scores in log_clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std(), params))
    print(scores)

best params: {'C': 1000, 'tol': 0.001}
-0.044 (+/-0.020) for {'C': 100, 'tol': 0.001}
[-0.04893948 -0.02490256 -0.07837384 -0.04507673 -0.02316519]
-0.044 (+/-0.020) for {'C': 100, 'tol': 0.0001}
[-0.04937707 -0.02511796 -0.07826285 -0.04507673 -0.02299002]
-0.041 (+/-0.023) for {'C': 1000, 'tol': 0.001}
[-0.04247012 -0.01781544 -0.08315391 -0.039078   -0.02200079]
-0.041 (+/-0.024) for {'C': 1000, 'tol': 0.0001}
[-0.04431737 -0.01847249 -0.08396315 -0.0421845  -0.01731101]


In [51]:
from sklearn.cross_validation import cross_val_score
scr = -cross_val_score(log_clf,train_x, train_y, scoring='neg_log_loss', cv = 5)
print(scr)

[ 0.04247012  0.01781544  0.08396315  0.0421845   0.02200079]


In [67]:
test_data = pd.read_csv('D:/kaggleCompetition/LeafClassification/test.csv/test.csv')
test_data.shape

(594, 193)

In [68]:
test_ids = test_data.pop('id')

In [54]:
test_x = test_data.values

In [56]:
#standard the test data
test_x = scaler.transform(test_x)
test_y = log_clf.predict_proba(test_x)

In [57]:
submission = pd.DataFrame(test_y, index=test_ids, columns=le.classes_)
submission.to_csv('D:/kaggleCompetition/LeafClassification/submission_logregression.csv')